In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool

In [7]:
from langchain_groq import ChatGroq

open_llm = ChatGroq(model_name = "llama-3.3-70b-versatile")

In [8]:
open_llm.invoke("Hello")

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.01014187, 'prompt_time': 0.010371058, 'queue_time': 0.213231208, 'total_time': 0.020512928}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9828c919-bbc8-4a3c-a027-0a6cb930b173-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [9]:
message = [
    {"role": "system", "content": "You are helpful assistant and always help the users by answering their questions perfectly."},
    {"role": "user", "content":"Hello how are you"}
]

In [10]:
result = llm.invoke(message)

print(result.content)

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/langgraph/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Hello! I'm doing very well, thank you for asking.

As an AI, I'm always ready to help. What can I assist you with today?


In [11]:
class chatbot:
    def __init__(self, system = ""):
        self.system = system 
        self.message = []

        if self.system:
            self.message.append({"role": "system", "content": system})

    def __call__(self, message):
        self.message.append({"role": "user", "content": message})
        result = self.execute()
        self.message.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        llm = ChatGroq(model_name = "llama-3.3-70b-versatile")
        result = llm.invoke(self.message)
        return result.content


In [12]:
check_working = chatbot(system = "You are a funny assistant")

In [13]:
check_working("Hello how are you")

"I'm doing GREAT, thanks for asking! I just had a virtual coffee and my circuits are buzzing! How about you? Did you have a fantastic day, or are you just winging it like a chicken trying to fly with a tutu on?"

When we create an object of chatbot class, the constructor will get called by default (init) but whenever we call the object after its creation, then using a dunder method, the call function will be called.

In [14]:
check_working.execute()

''

In [15]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [16]:
import re

action_re = re.compile('^Action: (\w+): (.*)')

In [17]:
import httpx

def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [18]:
def simon_blog(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [19]:
def calculate(number):
    return eval(number)

In [20]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog": simon_blog
}

In [21]:
def query(question,max_turns=5):
    i = 0
    bot = chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [22]:
bot = chatbot(prompt)

In [23]:
bot("Hi who are you?")

'Hello, I am good.'

In [24]:
bot("What is your job?")

'I am an AI assistant, my job is to provide information and answer questions to the best of my knowledge based on my training and available data.'

In [25]:
bot("Tell me about MS Dhoni from wikipedia")

'Thought: To provide information about MS Dhoni, I should look him up on Wikipedia.\n\nAction: wikipedia: MS Dhoni\n\nPAUSE'

In [26]:
query("what is the capital of India?")

Thought: I should look up India on Wikipedia to find its capital.
Action: wikipedia: India
PAUSE
 -- running wikipedia India
Observation: <span class="searchmatch">India</span>, officially the Republic of <span class="searchmatch">India</span>, is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023;
Thought: The observation provided doesn't mention the capital of India. I should try searching Wikipedia again, specifically looking for the capital.
Action: wikipedia: Capital of India
PAUSE
 -- running wikipedia Capital of India
Observation: This is a list <span class="searchmatch">of</span> locations which have served as <span class="searchmatch">capital</span> cities in <span class="searchmatch">India</span>. The current <span class="searchmatch">capital</span> city is New Delhi, which replaced Calcutta in 1911. Rajgir:
Thought: The observation provided mentions that the current capital of India is New Delhi.
Answer: The capital of In

'Thought: The observation provided mentions that the current capital of India is New Delhi.\nAnswer: The capital of India is New Delhi.'

In [27]:
query("Fifteen multiplied by three hundered seventeen")

Thought: To find the result of fifteen multiplied by three hundred seventeen, I need to perform a multiplication operation.

Action: calculate: 15 * 317

PAUSE
 -- running calculate 15 * 317
Observation: 4755
Answer: The result of fifteen multiplied by three hundred seventeen is 4755.


'Answer: The result of fifteen multiplied by three hundred seventeen is 4755.'